# Evaluation

Steps:
1. Harmonize the predictions to have the same format
2. Extract the frequencies using a sliding window approach
3. Evaluate the performance of the models
4. Visualize the results

## Step 1: Harmonize the predictions

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
raft_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
raft_predictions = pd.read_csv(raft_file)
raft_predictions['signal'] = raft_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
raft_predictions = raft_predictions[raft_predictions['roi'] == 'chest']

# Only keep the columns that are needed
raft_predictions = raft_predictions[['subject', 'setting', 'model', 'signal']]

raft_predictions.head()

In [ ]:
flownet_file = utils.join_paths(signals_dir, 'flownet_predictions.csv')
flownet_predictions = pd.read_csv(flownet_file)
flownet_predictions['signal'] = flownet_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
flownet_predictions = flownet_predictions[flownet_predictions['roi'] == 'chest']

# Only keep the columns that are needed
flownet_predictions = flownet_predictions[['subject', 'setting', 'model', 'signal']]

flownet_predictions.head()

In [ ]:
pretrained_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
pretrained_predictions = pd.read_csv(pretrained_file)
pretrained_predictions['signal'] = pretrained_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
pretrained_predictions = pretrained_predictions[['subject', 'setting', 'model', 'signal']]

pretrained_predictions.head()

In [ ]:
lucas_kanade_file = utils.join_paths(signals_dir, 'lucas_kanade.csv')
lucas_kanade = pd.read_csv(lucas_kanade_file)
lucas_kanade['signal'] = lucas_kanade['signal'].apply(eval).apply(np.array)

# Rename column method to model
lucas_kanade.rename(columns={'method': 'model'}, inplace=True)

# Remove all the rows that have a signal with a length of 0
lucas_kanade = lucas_kanade[lucas_kanade['grey'] == False]

# Only keep the columns that are needed
lucas_kanade = lucas_kanade[['subject', 'setting', 'model', 'signal']]

lucas_kanade.head()

In [ ]:
pixel_intensity_file = utils.join_paths(signals_dir, 'pixel_intensity.csv')
pixel_intensity = pd.read_csv(pixel_intensity_file)
pixel_intensity['signal'] = pixel_intensity['signal'].apply(eval).apply(np.array)

# Rename column method to model
pixel_intensity.rename(columns={'method': 'model'}, inplace=True)

# Only keep the columns that are needed
pixel_intensity = pixel_intensity[['subject', 'setting', 'model', 'signal']]

pixel_intensity.head()

In [ ]:
r_ppg_path = utils.join_paths(signals_dir, 'r_ppg_predictions.csv')

r_ppg_prediction = pd.read_csv(r_ppg_path)
r_ppg_prediction['signal'] = r_ppg_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
r_ppg_prediction = r_ppg_prediction[['subject', 'setting', 'model', 'signal']]
r_ppg_prediction.head()

In [ ]:
transformer_path = utils.join_paths(signals_dir, 'transformer_predictions.csv')

transformer_prediction = pd.read_csv(transformer_path)
transformer_prediction['signal'] = transformer_prediction['signal'].apply(eval).apply(np.array)

# Add a tf_ prefix to the model names
transformer_prediction['model'] = 'tf_' + transformer_prediction['model']

# Only keep the columns that are needed
transformer_prediction = transformer_prediction[['subject', 'setting', 'model', 'signal']]
transformer_prediction.head()

In [ ]:
#
# The random signal is used as a baseline to see how well the models perform against a random predictions
#
random_path = utils.join_paths(signals_dir, 'random_predictions.csv')

random_prediction = pd.read_csv(random_path)
random_prediction['signal'] = random_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
random_prediction = random_prediction[['subject', 'setting', 'model', 'signal']]
random_prediction.head()

In [ ]:
rhythm_former_path = utils.join_paths(signals_dir, 'rhythm_former.csv')

rhythm_former = pd.read_csv(rhythm_former_path)
rhythm_former['signal'] = rhythm_former['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
rhythm_former = rhythm_former[['subject', 'setting', 'model', 'signal']]
rhythm_former.head()

In [ ]:
predictions = pd.concat([
    raft_predictions,
    flownet_predictions,
    pretrained_predictions,
    lucas_kanade,
    pixel_intensity,
    r_ppg_prediction,
    transformer_prediction,
    random_prediction,
    rhythm_former,
])
len(predictions)

In [ ]:
# Show all models
predictions['model'].unique()

## Step 2: Extract the frequencies using a sliding window approach

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
subject = 'Proband23'
setting = '101_natural_lighting'

In [ ]:
sampling_rate = 30
lowpass = 0.1
highpass = 0.5

In [ ]:
from respiration.analysis import (
    butterworth_filter,
    normalize_signal,
    detrend_tarvainen,
)

from scipy.signal import detrend

models = [
    # 'lucas_kanade',
    'RF_20240802_155121',
    # 'RF_20240726_104536',
    # 'raft_small',
    # 'pixel_intensity_grey',
    # 'tf_20240729_195756',
    # 'MMPD_intra_RhythmFormer',
    # 'mtts_can',
    # 'big_small',
]

signals = []

for model in models:
    prediction = predictions[
        (predictions['subject'] == subject) &
        (predictions['setting'] == setting) &
        (predictions['model'] == model)].iloc[0]['signal']

    print(f'{model}: {prediction.shape}')

    # Normalize the signals
    prediction = normalize_signal(prediction)

    # Filter the signals
    prediction = butterworth_filter(prediction, sampling_rate, lowpass, highpass)

    # Add the signals to the list
    signals.append({
        'label': model,
        'signal': prediction,
    })

In [ ]:
# Get the ground truth signal
gt_signal = dataset.get_breathing_signal(subject, setting)
gt_signal = normalize_signal(gt_signal)
gt_signal = butterworth_filter(gt_signal, sampling_rate, lowpass, highpass)

In [ ]:
import matplotlib.pyplot as plt
from respiration.analysis import (
    find_crossing_points,
    find_crossing_points_nfcp,
    build_cross_curve,
)

from scipy.signal import find_peaks

plt.figure(figsize=(20, 5))
# plt.plot(gt_signal, label='gt', linestyle='--')

# Add crossing points
gt_signal_x = np.diff(gt_signal)

prominence = 0.5
distance = 0.1 * sampling_rate
# distance = None
gt_peaks, _ = find_peaks(gt_signal, prominence=prominence, distance=distance)
plt.scatter(gt_peaks, gt_signal[gt_peaks], color='blue')
print(f'peaks: {(len(gt_peaks) / (len(gt_signal) / sampling_rate)) * 60:.2f} bpm')

# crossing_points, _ = signal.find_peaks(-gt_signal, prominence=1.0)
# plt.scatter(crossing_points, gt_signal[crossing_points], color='blue')
# crossing_points = find_crossing_points(gt_signal_x)
# crossing_points = find_crossing_points_nfcp(gt_signal_x, sampling_rate)
# plt.scatter(crossing_points, gt_signal[crossing_points], color='blue')
# plt.plot(gt_signal_x, label='gt', linestyle='--')

# gt_signal_cross = build_cross_curve(gt_signal, sampling_rate)
# plt.plot(gt_signal_cross, label='gt_cross')

for signal in signals:
    # plt.plot(signal['signal'], label=signal['label'])
    peaks, _ = find_peaks(signal['signal'], prominence=prominence, distance=distance)

    random_color = np.random.rand(3, )
    plt.scatter(peaks, signal['signal'][peaks], color=random_color)
    print(f'{signal["label"]}: {(len(peaks) / (len(signal["signal"]) / sampling_rate)) * 60:.2f} bpm')

plt.legend()
plt.show()

In [ ]:
_, axs = plt.subplots(2, 1, figsize=(20, 10))

signal_x = signals[0]['signal']
gt_signal = gt_signal[:len(signal_x)]

peaks_gt, _ = find_peaks(gt_signal, prominence=prominence, distance=distance)
peaks_gt_x, _ = find_peaks(-gt_signal, prominence=prominence, distance=distance)
# peaks_x, _ = find_peaks(signal_x, prominence=prominence, distance=distance)

axs[0].plot(gt_signal)
axs[0].set_title('Ground Truth (GT)')
axs[0].scatter(peaks_gt, gt_signal[peaks_gt], color='blue')
axs[0].scatter(peaks_gt_x, gt_signal[peaks_gt_x], color='green')

# Add vertical lines every 30 seconds
for inx in range(0, len(gt_signal), sampling_rate * 30):
    axs[0].axvline(x=inx, color='red', linestyle='--')

axs[1].plot(signal_x)
axs[1].set_title(f'Predictions {signals[0]["label"]}')
axs[1].scatter(peaks_gt, signal_x[peaks_gt], color='blue')
axs[1].scatter(peaks_gt_x, signal_x[peaks_gt_x], color='green')

# Add vertical lines every 30 seconds
for inx in range(0, len(signal_x), sampling_rate * 30):
    axs[1].axvline(x=inx, color='red', linestyle='--')
    
# Add a legend
axs[0].legend(['signal', 'peaks', 'troughs'])
axs[1].legend(['signal', 'peaks-GT', 'troughs-GT'])